In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [31]:
# Define global variables.
hidden_size = 100
num_classes = 228
learning_rate = 3e-4
num_train_steps = 10
num_train_per_eval = 2

# input_fn arguments.
should_augment = False
images_limit = 1000  # How many images to train.
batch_size = 32
num_threads = 8
input_folder = '/home/shared/cs231n-fashion/data/train_processed'
label_json_path = '/home/shared/cs231n-fashion/data/train.json'

In [27]:
def model_fn(features, labels, mode):
    """Model function for fashion classes predictions.
    
    Inputs:
        features: dict.
            Required key: "image". The value needs to have shape (batch_size, 299, 299, 3),
                and each value needs to be in range [0, 1].
        labels: shape (batch_size, num_classes)
        mode: tf.estimator.ModeKeys.(PREDICT|TRAIN|EVAL)
    
    Returns:
        estimator_spec.
    """
    # class_prob > threshold will be outputted.
    thresholds =[0.3, 0.5, 0.7]
    threshold = 0.5
#     threshold = tf.get_variable('threshold_unbound', initializer=0.5)
#     threshold = tf.clip_by_value(threshold, 0.1, 0.9, name='threshold')
    
    # Input layer.
    images = features
    module = hub.Module("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/1")
    features = module(images)  # (batch_size, D)
    
    # Create multi-head sigmoid outputs.
    # It measures the independent probability of a class showing in the image.
    raw_logits = tf.contrib.layers.fully_connected(
        inputs=features,
        num_outputs=num_classes,
        activation_fn=None)  # (batch_size, num_classes)
    
    raw_probs = tf.sigmoid(raw_logits)  # (batch_size, num_classes)
    
#     # RNN layer.
#     gru_cell = tf.nn.rnn_cell.GRUCell(hidden_size)
#     outputs, _ = tf.nn.dynamic_rnn(cell=gru_cell, inputs=)
    
    predictions = {
        'classes': raw_probs > threshold,
        'probs': raw_probs,
    }
    
    # PREDICT mode.
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate loss (for both TRAIN and EVAL mode).
    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=labels,
        logits=raw_logits)
    
    # TRAIN mode.
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)    
    
    # Add evalutaion metrics (for EVAL mode).
    precisions = tf.metrics.precision_at_thresholds(
        labels=labels,
        predictions=predictions['probs'],
        thresholds=thresholds)
    recalls = tf.metrics.recall_at_thresholds(
        labels=labels,
        predictions=predictions['probs'],
        thresholds=thresholds)
    auc = tf.metrics.auc(
        labels=labels,
        predictions=predictions['probs'])
    
    eval_metric_ops = {
        'precisions': precisions,
        'recalls': recalls,
        'auc': auc,
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [28]:
# Create the estimator.
classifier = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir='../model_dir/baseline')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_dir/baseline', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0a62377898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
import input_fn

train_input_fn = lambda: input_fn.input_fn(
        input_folder,
        label_json_path,
        augment=should_augment,
        batch_size=batch_size,
        num_threads=num_threads,
        images_limit=images_limit)

In [ ]:
# Train the classifier.
# Run evaluation every num_train_per_eval training steps.
# Print the evalutation data and add the data into eval_data_list.

has_trained_steps = 0
eval_data_list = []

while has_trained_steps < num_train_steps:
    classifier.train(train_input_fn, steps=num_train_per_eval)
    
    # TODO: add eval_input_fn.
    eval_data = classifier.evaluate(train_input_fn, steps=1)
    print(eval_data)
    eval_data_list.append(eval_data)
    
    has_trained_steps += num_train_per_eval

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/weights:0 from checkpoint b'/tmp/tf

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_3b_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_3b_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_4a_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_4a_3x3/BatchNorm/beta
INFO:t

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/weights:0 from checkpoint b'/tmp/t

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_mo

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchN

...

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_19/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_19/Branch_1/Conv2d_0a_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_19/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_19/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_19/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_19/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_2/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_2/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_2/Branch_1/Conv2d_0c_7x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_2/Branch_1/Conv2d_0c_7x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_2/Conv2d_1x1/biases:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_2/Conv2d_1x1/biases
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_2/Conv2d_1x1/weights:0 from checkpoint b'/tmp/tfhub_modul

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d_0b_1x7/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d_0b_1x7/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_3/Branch_1/Conv2d

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_4/Conv2d_1x1/biases:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_4/Conv2d_1x1/biases
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_4/Conv2d_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_4/Conv2d_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_5/Branch_0/Conv2d_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_5/Branch_0/Conv2d_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_5/Branch_0/Conv2d_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a33748

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_1/block17_6/Branch_1/Conv2d_0b_1x7/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_1/block17_6/Branch_1/Conv2d_0b_1x7/BatchNorm/beta


In [29]:
# Example output for running evaluate function.
classifier.evaluate(train_input_fn, steps=1)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Block8/Branch_0/Conv2d_1x1/weights:0 from checkpoint b'/tmp/tf

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_3b_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_3b_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_3b_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Conv2d_4a_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Conv2d_4a_3x3/BatchNorm/beta
INFO:t

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_5b/Branch_2/Conv2d_0c_3x3/weights:0 from checkpoint b'/tmp/t

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Mixed_7a/Branch_0/Conv2d_1a_3x3/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_mo

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat/block35_1/Branch_1/Conv2d_0b_3x3/BatchN

...

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_4/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/blo

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_5/Branch_1/Conv2d_0c_3x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_5/Branch_1/Conv2d_0c_3x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_5/Branch_1/Conv2d_0c_3x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_5/Branch_1/Conv2d_0c_3x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_5/Conv2d_1x1/biases:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_5/Conv2d_1x1/biases
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_5/Conv2d_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33c

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0b_1x3/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0b_1x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_7/Branch_1/Conv2d_0b_1x3

INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_8/Conv2d_1x1/biases:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_8/Conv2d_1x1/biases
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_8/Conv2d_1x1/weights:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_8/Conv2d_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_9/Branch_0/Conv2d_1x1/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f843fca5/variables/variables' with InceptionResnetV2/Repeat_2/block8_9/Branch_0/Conv2d_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionResnetV2/Repeat_2/block8_9/Branch_0/Conv2d_1x1/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/b33cf40b3b329fdcead0a337480d32b1f

{'auc': 0.5422916,
 'loss': 0.6782205,
 'precisions': array([0.02673003, 0.02917342, 0.03501945], dtype=float32),
 'recalls': array([0.95238096, 0.47619048, 0.04761905], dtype=float32),
 'global_step': 1}

In [20]:
# Test cell to validate the input_fn.
# The output batch_images.shape should be (batch_size, 299, 299, 3)
#            batch_labels.shape should be (batch_size, 228)

should_augment = False
images_limit = 4  # How many images to train.
batch_size = 2
num_threads = 2
input_folder = '/home/shared/cs231n-fashion/data/train_processed'
label_json_path = '/home/shared/cs231n-fashion/data/train.json'

with tf.Session() as sess:
    next_batch = input_fn.input_fn(
        input_folder,
        label_json_path,
        augment=should_augment,
        batch_size=batch_size,
        num_threads=num_threads,
        images_limit=images_limit)
    for _ in range(5):
        batch_images, batch_labels = sess.run(next_batch)
        print(batch_images.shape, batch_labels.shape)

(2, 299, 299, 3) (2, 228)
(2, 299, 299, 3) (2, 228)
(2, 299, 299, 3) (2, 228)
(2, 299, 299, 3) (2, 228)
(2, 299, 299, 3) (2, 228)
